In [2]:
!pip install -q streamlit pyngrok mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.7 MB/s eta 0:00:00


In [3]:
!ngrok authtoken 2qaPM3tRl12NtJ2mBMRp5QnQItc_6H7Du3N9UkuPCJcWs3UAt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# 🔥 **Testing Database**

In [4]:
import mysql.connector
from mysql.connector import Error

hostname = "dh726.h.filess.io"
database = "Testing_oxygenlove"
port = 61001  # This should be an integer
username = "Testing_oxygenlove"
password = "7aa784e9040d7312bd35fcfe577ad9bdf7553593"

try:
    connection = mysql.connector.connect(
        host=hostname,
        database=database,
        user=username,
        password=password,
        port=port
    )
    if connection.is_connected():
        print("✅ Connected to MySQL server")
        cursor = connection.cursor()
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()
        print("Tables in database:", tables)

except Error as e:
    print("❌ Error connecting to MySQL:", e)

✅ Connected to MySQL server
Tables in database: [('books',), ('transactions',), ('users',)]


In [7]:
def get_connection():
    return mysql.connector.connect(
        host="dh726.h.filess.io",
        database="Testing_oxygenlove",
        user="Testing_oxygenlove",
        password="7aa784e9040d7312bd35fcfe577ad9bdf7553593",
        port=61001
    )

def check_constraint_name():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT CONSTRAINT_NAME
        FROM information_schema.KEY_COLUMN_USAGE
        WHERE TABLE_NAME = 'transactions' AND COLUMN_NAME = 'buku_id';
    """)
    for row in cursor.fetchall():
        print("Found constraint:", row[0])
    cursor.close()
    conn.close()

check_constraint_name()

Found constraint: fk_buku_id


In [8]:
def show_transactions_for_book(book_id):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM transactions WHERE buku_id = %s", (book_id,))
    rows = cursor.fetchall()
    print(f"Transactions for book ID {book_id}:")
    for row in rows:
        print(row)
    cursor.close()
    conn.close()

show_transactions_for_book(18829)

Transactions for book ID 18829:
(5, 18829, 2008561033, datetime.datetime(2025, 6, 2, 16, 39, 46), datetime.datetime(2025, 6, 2, 16, 53, 7))
(7, 18829, 2008561055, datetime.datetime(2025, 6, 2, 16, 44, 56), datetime.datetime(2025, 6, 2, 16, 53, 59))
(8, 18829, 2008561033, datetime.datetime(2025, 6, 2, 16, 54, 45), datetime.datetime(2025, 6, 2, 16, 55, 26))
(9, 18829, 2008561055, datetime.datetime(2025, 6, 3, 0, 43, 38), datetime.datetime(2025, 6, 3, 0, 52, 25))
(11, 18829, 2008561033, datetime.datetime(2025, 6, 4, 3, 28, 56), datetime.datetime(2025, 6, 4, 3, 39, 8))
(14, 18829, 2008561033, datetime.datetime(2025, 6, 5, 1, 54, 59), datetime.datetime(2025, 6, 5, 1, 55, 19))
(15, 18829, 2008561033, datetime.datetime(2025, 6, 5, 1, 57, 33), datetime.datetime(2025, 6, 5, 2, 1, 32))


#🎯 **Real Project Using Streamlit**

In [9]:
%%writefile dashboard.py

import streamlit as st
import mysql.connector
import pandas as pd
import mysql.connector
from urllib.parse import quote
import random
import math

# --- DB Connection ---
def get_connection():
    return mysql.connector.connect(
        host="dh726.h.filess.io",
        database="Testing_oxygenlove",
        user="Testing_oxygenlove",
        password="7aa784e9040d7312bd35fcfe577ad9bdf7553593",
        port=61001
    )

def fetch_books():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books")
    rows = cursor.fetchall()
    cols = [desc[0] for desc in cursor.description]
    conn.close()
    return pd.DataFrame(rows, columns=cols)

def get_users():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    rows = cursor.fetchall()
    cols = [desc[0] for desc in cursor.description]
    conn.close()
    return pd.DataFrame(rows, columns=cols)

@st.dialog("🏹 Add New Users")
def add_users():
    name = st.text_input("Fullname: ")
    user_id = st.text_input("User ID: ")
    if st.button("Submit"):
        if name and user_id:
            conn = get_connection()
            cursor = conn.cursor()
            cursor.execute("INSERT INTO users (id, nama) VALUES (%s, %s)", (user_id, name))
            conn.commit()
            cursor.close()
            conn.close()
            st.toast(f"User {name} with ID {user_id} success added!", icon='🎉')
            st.rerun()
        else:
            st.warning("Please fill out all fields.")


# --- Streamlit App ---
st.title("📚 Library Borrow System")

# Add new books
for key, default in {
    "show_add_form": False,
    "form_submitted": False,
}.items():
    if key not in st.session_state:
        st.session_state[key] = default

if st.session_state.form_submitted:
    # Reset only control values
    st.session_state.show_add_form = False
    st.session_state.form_submitted = False

    # Clear the input widgets by resetting Streamlit keys
    st.session_state.new_book_id = ""
    st.session_state.new_book_title = ""
    st.session_state.new_book_author = ""
    st.session_state.new_book_status = "Available"
    st.rerun()

with st.expander("➕ Add New Book", expanded=st.session_state.show_add_form):

    if not st.session_state.show_add_form:
      st.session_state.show_add_form = True

    new_book_id = st.text_input("Book ID", key="book_id")
    new_book_title = st.text_input("Book Title", key="book_title")
    new_book_author = st.text_input("Book Author", key="book_author")
    book_status_options = ["Available", "Borrowed"]
    new_book_status = st.selectbox(
      "Book Status",
      options=book_status_options,
      index=book_status_options.index(st.session_state.get("book_status", "Available")),
      key="book_status"
    )

    if st.button("Add Book"):
        if new_book_id and new_book_title and new_book_author:
            status_value = 1 if new_book_status == "Available" else 0
            conn = get_connection()
            cursor = conn.cursor()
            try:
                cursor.execute(
                    "INSERT INTO books (id, judul, penulis, status) VALUES (%s, %s, %s, %s)",
                    (
                        new_book_id,
                        new_book_title,
                        new_book_author,
                        status_value,
                    )
                )
                conn.commit()
                st.success(f"✅ Book '{new_book_title}' added successfully!")

                # Clear form state after successful submission
                st.session_state.form_submitted = True
                st.rerun()

            except mysql.connector.Error as e:
                st.error(f"❌ Error adding book: {e}")

            cursor.close()
            conn.close()
        else:
            st.warning("❌ Please fill in all fields.")

# View All Books
st.header("📋 Book List")
books_df = fetch_books()
books_df["status"] = books_df["status"].map({1: "Available", 0: "Borrowed"})

if "edit_id" not in st.session_state:
    st.session_state.edit_id = None

for index, row in books_df.iterrows():
    cols = st.columns([1.5, 3, 3, 2, 3, 3])

    cols[0].write(row["id"])
    cols[1].write(row["judul"])
    cols[2].write(row["penulis"])
    cols[3].write(row["status"])

    if cols[4].button("🧺 Hapus", key=f"delete_{row['id']}"):
        conn = get_connection()
        cursor = conn.cursor()

        # Check if book is currently borrowed (status=0 or active transactions without return)
        cursor.execute("SELECT COUNT(*) FROM transactions WHERE buku_id = %s", (row['id'],))
        active_borrow_count = cursor.fetchone()[0]

        if active_borrow_count > 0:
            st.warning(f"⚠️ Cannot delete '{row['judul']}' because it has borrowing history!")
        else:
            cursor.execute("DELETE FROM books WHERE id = %s", (row['id'],))
            conn.commit()
            st.success(f"✅ Book '{row['judul']}' deleted successfully!")
            st.rerun()

        cursor.close()
        conn.close()

    if cols[5].button("✏️ Edit", key=f"edit_{row['id']}"):
        # Check if book is currently borrowed
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM transactions WHERE buku_id = %s AND tanggal_kembali IS NULL", (row['id'],))
        active_borrow_count = cursor.fetchone()[0]
        cursor.close()
        conn.close()

        if active_borrow_count > 0:
            st.warning(f"⚠️ Cannot edit '{row['judul']}' because it is currently borrowed!")
        else:
            st.session_state.edit_id = row["id"]

        cursor.close()
        conn.close()

    if st.session_state.edit_id == row["id"]:
        st.subheader(f"✏️ Editing Book ID: {row['id']}")
        new_title = st.text_input("New Title", value=row["judul"], key=f"title_{row['id']}")
        new_author = st.text_input("New Author", value=row["penulis"], key=f"author_{row['id']}")

        if st.button("💾 Save Changes", key=f"save_{row['id']}"):
            conn = get_connection()
            cursor = conn.cursor()
            cursor.execute("UPDATE books SET judul = %s, penulis = %s WHERE id = %s", (new_title, new_author, row["id"]))
            conn.commit()
            cursor.close()
            conn.close()
            st.success(f"✅ Book '{new_title}' updated successfully!")
            st.session_state.edit_id = None
            st.rerun()

# Users Actions Example
st.header("🤼 Users Example")
users_df = get_users()
default_icon = "💂🏻‍♀️"
users_df["icon"] = default_icon
users_df["display"] = users_df["icon"] + " " + users_df["nama"]

# Init session state
if "selected_user_id" not in st.session_state:
    st.session_state.selected_user_id = None
if "selected_user_display" not in st.session_state:
    st.session_state.selected_user_display = ""
if "show_add_form_2" not in st.session_state:
    st.session_state.show_add_form_2 = False

# Layout users + add user button
num_columns = 3
total_buttons = len(users_df) + 1
num_rows = math.ceil(total_buttons / num_columns)

user_index = 0
for row_num in range(num_rows):
    cols = st.columns(num_columns)
    for col_num in range(num_columns):
        if user_index < len(users_df):
            row = users_df.iloc[user_index]
            with cols[col_num]:
                if st.button(row["display"], key=f"user_{row['id']}"):
                    st.session_state.selected_user_id = row.id
                    st.session_state.selected_user_display = row.display
            user_index += 1
        elif user_index == len(users_df):
            with cols[col_num]:
                if st.button("➕ Add User", key="show_add_user_btn"):
                    st.session_state.show_add_form = True
            user_index += 1

# Add Users Form
if st.session_state.show_add_form_2:
    add_users()
    st.session_state.show_add_form_2 = False

if st.session_state.selected_user_id:
    st.write(f"👤 Selected user: {st.session_state.selected_user_display}")
    user_id = int(st.session_state.selected_user_id)

    conn = get_connection()
    cursor = conn.cursor()

    tab1, tab2 = st.tabs(["📤 Return Book", "📥 Borrow Book"])

    with tab1:
        cursor.execute("""
            SELECT b.id, b.judul FROM books b
            JOIN transactions t ON b.id = t.buku_id
            WHERE t.user_id = %s AND t.tanggal_kembali IS NULL
        """, (user_id,))
        borrowed_books = cursor.fetchall()

        if borrowed_books:
            selected_return = st.selectbox("Select a book to return:", borrowed_books, format_func=lambda x: x[1])

            if st.button("🔄 Confirm Return", key="confirm_return", disabled=selected_return is None):
                cursor.execute("""
                    UPDATE transactions
                    SET tanggal_kembali = NOW()
                    WHERE buku_id = %s AND user_id = %s AND tanggal_kembali IS NULL
                """, (selected_return[0], user_id))
                cursor.execute("UPDATE books SET status = 1 WHERE id = %s", (selected_return[0],))
                conn.commit()
                st.success(f"✅ Book '{selected_return[1]}' returned successfully!")
                st.session_state.selected_user_id = None
                st.rerun()
        else:
            st.info("📚 No borrowed books to return.")

    with tab2:
        cursor.execute("SELECT id, judul FROM books WHERE status = 1")
        available_books = cursor.fetchall()

        if available_books:
            selected_borrow = st.selectbox("Select a book to borrow:", available_books, format_func=lambda x: x[1])

            if st.button("📩 Confirm Borrow", key="confirm_borrow", disabled=selected_borrow is None):
                cursor.execute("""
                    INSERT INTO transactions (buku_id, user_id, tanggal_pinjam)
                    VALUES (%s, %s, NOW())
                """, (selected_borrow[0], user_id))
                cursor.execute("UPDATE books SET status = 0 WHERE id = %s", (selected_borrow[0],))
                conn.commit()
                st.success(f"✅ Book '{selected_borrow[1]}' borrowed successfully!")
                st.session_state.selected_user_id = None
                st.rerun()
        else:
            st.warning("❌ No books available for borrowing.")

    cursor.close()
    conn.close()

Overwriting dashboard.py


In [5]:
from pyngrok import ngrok
import subprocess
import time
import os

# Kill any previous tunnels
ngrok.kill()

# Set port
port = 8501

# Start Streamlit server in background
print("Starting Streamlit server...")
process = subprocess.Popen(["streamlit", "run", "dashboard.py", "--server.port", str(port)])

# Wait to give Streamlit time to start
time.sleep(10)

# Open ngrok tunnel
print("Starting ngrok tunnel...")
public_url = ngrok.connect(port, "http").public_url
print("✅ Streamlit is live at:", public_url)

Starting Streamlit server...
Starting ngrok tunnel...
✅ Streamlit is live at: https://edce-34-106-97-131.ngrok-free.app


In [ ]:
!ps aux | grep ngrok

root         830  0.8  0.2 1254568 31644 ?       Sl   14:24   0:00 /root/.config/ngrok/ngrok start --none --log stdout
root        1025  0.0  0.0   7376  3536 ?        S    14:25   0:00 /bin/bash -c ps aux | grep ngrok
root        1027  0.0  0.0   6484  2304 ?        S    14:25   0:00 grep ngrok


In [ ]:
ngrok.kill()
!ps aux | grep ngrok

root        1044  0.0  0.0   7376  3588 ?        S    14:25   0:00 /bin/bash -c ps aux | grep ngrok
root        1046  0.0  0.0   6484  2356 ?        S    14:25   0:00 grep ngrok
